# Hyper-parameter optimization:

This notebook focuses on hyper-parameter optimization for the `MLPRegressor` model. 

In [4]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, csc_matrix
import logging
import time
from functools import partial
from hyperopt import hp,fmin,tpe,Trials
from hyperopt.pyll.base import scope
import price_alchemy.data_preprocessing as dp
import price_alchemy.config as cfg
import price_alchemy.logging_setup as ls
import price_alchemy.data_loading as dl
import price_alchemy.model_dispatcher as md
import price_alchemy.hpo as hpo
import price_alchemy.train as tr
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error

In [2]:
cd ..

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy


/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
# df= dl.load_data_sql(cfg.MYSQL_PASSWORD)

In [5]:
# in case of no internet
df= pd.read_csv('data/train.csv')

In [7]:
df_sample= dp.sample_df(df, sample_size=2000)

In [8]:
text_prep= cfg.TEXT_PREP_OPTS['spacy']
col_trans= 'tfidf_concat'

X,y= dp.preprocessing_pipe(df_sample, text_prep, cfg.COL_TRANS_OPTS[col_trans])

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/dags/price_alchemy/data_preprocessing.py:142: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['category_name'].replace('', np.nan, inplace=True)
/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/dags/price_alchemy/data_preprocessing.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [9]:
model_dispatch= md.models['mlp']

In [10]:
# define optimization function
optmization_function=partial(hpo.optimize,
                            X=X,
                            y=y.values,
                            model=model_dispatch)

# define trials 
trials=Trials()

result=fmin(fn=optmization_function,
        space=md.PARAMS,
        algo=tpe.suggest,
        max_evals=15,
        trials=trials,
        )

  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `

  7%|▋         | 1/15 [00:12<02:56, 12.60s/trial, best loss: 0.7416269555696863]

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `

 13%|█▎        | 2/15 [00:29<03:18, 15.27s/trial, best loss: 0.7416269555696863]

...

 80%|████████  | 12/15 [02:02<00:21,  7.02s/trial, best loss: 0.7012379880169652]

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `

 93%|█████████▎| 14/15 [02:27<00:10, 10.25s/trial, best loss: 0.7012379880169652]

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(

/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `

100%|██████████| 15/15 [02:32<00:00, 10.15s/trial, best loss: 0.7012379880169652]


What are the optimized parameters?

In [11]:
result

{'batch_size': 168,
 'hidden_layers': 5.0,
 'hidden_neurons': 10.0,
 'learning_rate': 0,
 'learning_rate_init': 0.04557585614660601,
 'max_iter': 187.0}

Retrain the model

In [15]:
lr_type=["invscaling","adaptive"]

ps= { 
    'hidden_layer_sizes': tuple([int(result['hidden_neurons'])] * int(result['hidden_layers'])),
    "max_iter":int(result["max_iter"]),
    "learning_rate_init": float(result["learning_rate_init"]),
    "batch_size":int(result["batch_size"]),
    "learning_rate":lr_type[int(result["learning_rate"])]
    }

print(ps)

# set optimized parameters
model_dispatch.set_params(**ps)

# train the model
model,metrics= tr.train_model(X, y.values, model=model_dispatch)

{'hidden_layer_sizes': (10, 10, 10, 10, 10), 'max_iter': 187, 'learning_rate_init': 0.04557585614660601, 'batch_size': 168, 'learning_rate': 'invscaling'}


/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/Users/mehuljain/Documents/course_related/ML_Ops/project/Price_Alchemy/.conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `bat

Print the metrics from the cross-validation

In [16]:
metrics

{'mse': 632.2259892887663,
 'rmse': 24.048055477678677,
 'r_2': -0.022787995400798945,
 'rmsle': 0.7012379880169652}